In [ ]:
!pip install deeptrack
!pip install ipython-autotime
!pip install sklearn
%load_ext autotime

In [ ]:
import numpy as np
import tensorflow as tf
import deeptrack as dt
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Reshape, UpSampling2D, Input, BatchNormalization, LayerNormalization, Activation, Layer, InputLayer, concatenate, Add, Dropout, LSTM

# **a)**

In [ ]:
particle = dt.Sphere(
    intensity=lambda: 10 + 10 * np.random.rand(),
    radius=lambda: MIN_SIZE + np.random.rand() * (MAX_SIZE - MIN_SIZE),
    position=lambda: IMAGE_SIZE * np.random.rand(2),
    vel=lambda: MAX_VEL * np.random.rand(2),
    position_unit="pixel",
)

train_data_a = []
for i in range(10000):
  sample = optics(particle) >> dt.Gaussian(sigma=0.1)
  train_data_a.append(sample.update()())

val_data_a = []
for i in range(2000):
  sample = optics(particle) >> dt.Gaussian(sigma=0.1)
  val_data_a.append(sample.update()())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# np.save("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/3a/train_data_a.npy", train_data_a)
# np.save("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/3a/val_data_a.npy", val_data_a)

In [ ]:
train_data_a = np.load("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/3a/train_data_a.npy")
val_data_a = np.load("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/3a/val_data_a.npy")

for i in range(10000):
  train_data_a[i] = train_data_a[i] / np.max(train_data_a[i])

for i in range(2000):
  val_data_a[i] = val_data_a[i] / np.max(val_data_a[i])

In [ ]:
bottle_nodes = 10

encoder = tf.keras.models.Sequential()
encoder.add(Conv2D(32, (3,3), (1,1), activation="relu", padding="same", input_shape=(64,64,1)))
encoder.add(MaxPooling2D((2,2), (2,2)))

encoder.add(Conv2D(64, (3,3), (1,1), activation="relu", padding="same"))
encoder.add(MaxPooling2D((2,2), (2,2)))

encoder.add(Conv2D(128, (3,3), (1,1), activation="relu", padding="same"))
encoder.add(MaxPooling2D((2,2), (2,2)))

encoder.add(Flatten())
encoder.add(Dense(128, activation="relu"))
encoder.add(Dense(bottle_nodes))



decoder = tf.keras.models.Sequential()
decoder.add(Dense(128, activation="relu"))
decoder.add(Dense(8*8*128, activation="relu"))
decoder.add(Reshape((8,8,128)))

decoder.add(UpSampling2D((2,2)))
decoder.add(Conv2D(128, (3,3), (1,1), activation="relu", padding="same"))

decoder.add(UpSampling2D((2,2)))
decoder.add(Conv2D(64, (3,3), (1,1), activation="relu", padding="same"))

decoder.add(UpSampling2D((2,2)))
decoder.add(Conv2D(32, (3,3), (1,1), activation="relu", padding="same"))
decoder.add(Conv2D(1, (3,3), (1,1), padding="same")) #change to linear

input=Input((64,64,1))
autoencoder = tf.keras.models.Model(inputs=input, outputs=decoder(encoder(input)))
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
autoencoder.compile(optimizer=optimizer, loss="mse")

autoencoder.summary()
# (I+P-K+S)/S

In [ ]:
test = np.array(train_data_a[0])
plt.figure()
plt.imshow(test)

test = tf.expand_dims(test, axis=0)
test_pred = autoencoder.predict(test, verbose=0)
plt.figure()
plt.imshow(test_pred[0])

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)
history = autoencoder.fit(x=np.array(train_data_a), y=np.array(train_data_a), validation_data=(np.array(val_data_a), np.array(val_data_a)), callbacks = [callback], epochs=20, verbose=1)

In [ ]:
# encoder.save("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/encoder.h5") # min loss 1*e-4,  latent space 4
# decoder.save("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/decoder.h5")
# autoencoder.save("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/autoencoder.h5")

In [ ]:
# encoder.save("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/encoder_b.h5") # min loss 6*e-5,  latent space 10
# decoder.save("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/decoder_b.h5")
# autoencoder.save("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/autoencoder_b.h5")

In [ ]:
# epochs = history.epoch
# train_loss = history.history['loss']
# val_loss = history.history['val_loss']

# plt.figure()
# plt.plot(epochs, train_loss, label="Training loss")
# plt.plot(epochs, val_loss, label="Validation loss")
# plt.legend(loc="upper right", fontsize=10)
# plt.xlabel("Epoch")
# plt.ylabel("Loss")
# plt.xlim([epochs[0], epochs[-1]])
# title = "loss.png".format(bottle_nodes)
# # plt.savefig(location+title)

test = np.array(train_data_a[149])#14, 100, 11
plt.figure()
plt.imshow(test)
title = "3a_n={}_no.1_TRUE.png".format(bottle_nodes)
# plt.savefig(location+title)

test = tf.expand_dims(test, axis=0)
test_pred = autoencoder.predict(test)
plt.figure()
plt.imshow(test_pred[0])
title = "3a_n={}_no.1__PRED.png".format(bottle_nodes)
# plt.savefig(location+title)

for seq in dataset_b:
  seq_b = seq[0]
  break

test = tf.expand_dims(seq_b, axis=0)
test = test / np.max(test)
test_pred = autoencoder.predict(test)
plt.figure()
plt.imshow(test_pred[0])
title = "3a_n={}_no.1__PRED.png".format(bottle_nodes)
# plt.savefig(location+title)

# **b)**

In [ ]:
bottle_nodes = 10
encoder = tf.keras.models.load_model("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/encoder_b.h5")
decoder = tf.keras.models.load_model("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/decoder_b.h5")
autoencoder = tf.keras.models.load_model("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/autoencoder_b.h5")

In [ ]:
IMAGE_SIZE = 64
sequence_length = 10  # Number of frames per sequence
MIN_SIZE = 0.5e-6
MAX_SIZE = 1.5e-6
MAX_VEL = 10  # Maximum velocity. The higher the trickier!
MAX_PARTICLES = 3  # Max number of particles in each sequence. The higher the trickier!

# Defining properties of the particles
particle = dt.Sphere(
    intensity=lambda: 10 + 10 * np.random.rand(),
    radius=lambda: MIN_SIZE + np.random.rand() * (MAX_SIZE - MIN_SIZE),
    position=lambda: IMAGE_SIZE * np.random.rand(2),
    vel=lambda: MAX_VEL * np.random.rand(2),
    position_unit="pixel",
)

# Defining an update rule for the particle position
def get_position(previous_value, vel):

    newv = previous_value + vel
    for i in range(2):
        if newv[i] > 63:
            newv[i] = 63 - np.abs(newv[i] - 63)
            vel[i] = -vel[i]
        elif newv[i] < 0:
            newv[i] = np.abs(newv[i])
            vel[i] = -vel[i]
    return newv


particle = dt.Sequential(particle, position=get_position)

# Defining properties of the microscope
optics = dt.Fluorescence(
    NA=1,
    output_region=(0, 0, IMAGE_SIZE, IMAGE_SIZE),
    magnification=10,
    resolution=(1e-6, 1e-6, 1e-6),
    wavelength=633e-9,
)

# Combining everything into a dataset.
# Note that the sequences are flipped in different directions, so that each unique sequence defines
# in fact 8 sequences flipped in different directions, to speed up data generation
sequential_images = dt.Sequence(
    optics(particle ** (lambda: 1 + np.random.randint(MAX_PARTICLES))),
    sequence_length=sequence_length,
)
dataset_b = sequential_images >> dt.FlipUD() >> dt.FlipDiagonal() >> dt.FlipLR()

In [ ]:
encoder_out_list = []
encoder_label_list = []

c = 0
for seq in dataset_b:

  train_seq = tf.convert_to_tensor(seq[:sequence_length-1]).numpy()
  for i in range(train_seq.shape[0]):
    train_seq[i] = train_seq[i] / np.max(train_seq[i])

  label = tf.expand_dims(np.array(seq[sequence_length-1]), 0).numpy()
  label = label / np.max(label)

  encoder_out = encoder.predict(train_seq, verbose=0)
  encoder_label = encoder.predict(label, verbose=0)[0]

  encoder_out_list.append(encoder_out)
  encoder_label_list.append(encoder_label)

  c += 1
  if c % 10 == 0:
    print(c, encoder_out.shape)

  if c % 100 == 0:
    encoder_outs = np.stack(encoder_out_list)
    encoder_labels = np.stack(encoder_label_list)
    print("Inputs: ", encoder_outs.shape, "Labels: ", encoder_labels.shape)

    # np.save("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/3b/encoder_outs_10.npy", encoder_outs)
    # np.save("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/3b/encoder_labels_10.npy", encoder_labels)

In [ ]:
# encoder_outs = np.load("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/3b/encoder_outs.npy")
# encoder_labels = np.load("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/3b/encoder_labels.npy")
# print(encoder_outs.shape, encoder_labels.shape)

encoder_outs = np.load("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/3b/encoder_outs_10.npy")
encoder_labels = np.load("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/3b/encoder_labels_10.npy")
print(encoder_outs.shape, encoder_labels.shape)

x_train, x_valid, y_train, y_valid = train_test_split(encoder_outs, encoder_labels, test_size=0.2, shuffle= True)
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape)

In [ ]:
class Time2Vector(Layer):
  def __init__(self, seq_len, **kwargs):
    super(Time2Vector, self).__init__()
    self.seq_len = seq_len

  def build(self, input_shape):
    self.weights_linear = self.add_weight(name='weight_linear', shape=(int(self.seq_len),), initializer='uniform', trainable=True)
    self.bias_linear = self.add_weight(name='bias_linear', shape=(int(self.seq_len),), initializer='uniform', trainable=True)
    self.weights_periodic = self.add_weight(name='weight_periodic', shape=(int(self.seq_len),), initializer='uniform', trainable=True)
    self.bias_periodic = self.add_weight(name='bias_periodic', shape=(int(self.seq_len),), initializer='uniform', trainable=True)

  def call(self, x):
    x = tf.math.reduce_mean(x[:,:,:], axis=-1)
    time_linear = self.weights_linear * x + self.bias_linear
    time_linear = tf.expand_dims(time_linear, axis=-1)
    time_periodic = tf.math.sin(tf.multiply(x, self.weights_periodic) + self.bias_periodic)
    time_periodic = tf.expand_dims(time_periodic, axis=-1)
    return tf.concat([time_linear, time_periodic], axis=-1)


class SingleAttention(Layer):
  def __init__(self, d_k, d_v):
    super(SingleAttention, self).__init__()
    self.d_k = d_k
    self.d_v = d_v

  def build(self, input_shape):
    self.query = Dense(self.d_k, input_shape=input_shape, kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform')
    self.key = Dense(self.d_k, input_shape=input_shape, kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform')
    self.value = Dense(self.d_v, input_shape=input_shape, kernel_initializer='glorot_uniform', bias_initializer='glorot_uniform')

  def call(self, inputs): # inputs = (in_seq, in_seq, in_seq)
    q = self.query(inputs[0])
    k = self.key(inputs[1])

    attn_weights = tf.matmul(q, k, transpose_b=True)
    attn_weights = tf.map_fn(lambda x: x/np.sqrt(self.d_k), attn_weights)
    attn_weights = tf.nn.softmax(attn_weights, axis=-1)

    v = self.value(inputs[2])
    attn_out = tf.matmul(attn_weights, v)
    return attn_out

In [ ]:
# Freeze weights of autoencoder
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
for k,v in autoencoder._get_trainable_state().items():
    k.trainable = False
autoencoder.compile(optimizer=optimizer, loss="mse")

for k,v in encoder._get_trainable_state().items():
    k.trainable = False
encoder.compile(optimizer=optimizer, loss="mse")

for k,v in decoder._get_trainable_state().items():
    k.trainable = False
decoder.compile(optimizer=optimizer, loss="mse")



# Transformer encoder
input = Input(shape=(sequence_length-1, bottle_nodes,))

time_vec = Time2Vector(sequence_length-1)(input)
embedding = concatenate([input, time_vec])
embedding_input = (embedding, embedding, embedding)
d = embedding.shape[-1]

attention1 = SingleAttention(d, d)(embedding_input)
attention2 = SingleAttention(d, d)(embedding_input)
attention3 = SingleAttention(d, d)(embedding_input)

attention_concat = concatenate([attention1, attention2, attention3])
multihead = Dense(embedding.shape[-1], activation='relu')(attention_concat)


dropout = Dropout(0.5)(multihead)
add_norm = LayerNormalization()(Add()([embedding, dropout]))

linear = Dense(d, activation='relu')(add_norm)
linear = Dense(d)(linear)

dropout = Dropout(0.5)(linear)
add_norm = LayerNormalization()(Add()([embedding, dropout]))






embedding_input = (add_norm, add_norm, add_norm)
attention1 = SingleAttention(d, d)(embedding_input)
attention2 = SingleAttention(d, d)(embedding_input)
attention3 = SingleAttention(d, d)(embedding_input)

attention_concat = concatenate([attention1, attention2, attention3])
multihead = Dense(embedding.shape[-1], activation='relu')(attention_concat)


dropout = Dropout(0.5)(multihead)
add_norm = LayerNormalization()(Add()([embedding, dropout]))

linear = Dense(d, activation='relu')(add_norm)
linear = Dense(d)(linear)

dropout = Dropout(0.5)(linear)
add_norm = LayerNormalization()(Add()([embedding, dropout]))





embedding_input = (add_norm, add_norm, add_norm)
attention1 = SingleAttention(d, d)(embedding_input)
attention2 = SingleAttention(d, d)(embedding_input)
attention3 = SingleAttention(d, d)(embedding_input)

attention_concat = concatenate([attention1, attention2, attention3])
multihead = Dense(embedding.shape[-1], activation='relu')(attention_concat)


dropout = Dropout(0.5)(multihead)
add_norm = LayerNormalization()(Add()([embedding, dropout]))

linear = Dense(d, activation='relu')(add_norm)
linear = Dense(d)(linear)

dropout = Dropout(0.5)(linear)
add_norm = LayerNormalization()(Add()([embedding, dropout]))






# embedding_input = (add_norm, add_norm, add_norm)
# attention1 = SingleAttention(d, d)(embedding_input)
# attention2 = SingleAttention(d, d)(embedding_input)
# attention3 = SingleAttention(d, d)(embedding_input)

# attention_concat = concatenate([attention1, attention2, attention3])
# multihead = Dense(embedding.shape[-1], activation='relu')(attention_concat)


# dropout = Dropout(0.5)(multihead)
# add_norm = LayerNormalization()(Add()([embedding, dropout]))

# linear = Dense(d, activation='relu')(add_norm)
# linear = Dense(d)(linear)

# dropout = Dropout(0.5)(linear)
# add_norm = LayerNormalization()(Add()([embedding, dropout]))







linear = Flatten()(add_norm)
linear = Dense(64, activation='relu')(linear)
linear = Dense(64, activation='relu')(linear)
linear = Dense(64, activation='relu')(linear)

linear = Dense(bottle_nodes)(linear)


transformer = tf.keras.models.Model(input, linear)
transformer.compile(loss="mse", optimizer=optimizer)
transformer.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)
history = transformer.fit(x_train, y_train, epochs=40, verbose=1, validation_data=(x_valid, y_valid), callbacks=[callback])

In [ ]:
# epochs = history.epoch
num_epochs = np.linspace(1,20,20)
train_loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure()
plt.plot(epochs, train_loss, label="Training loss")
plt.plot(epochs, val_loss, label="Validation loss")
plt.legend(loc="upper right", fontsize=10)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.xlim([epochs[0], epochs[-1]])
# title = "loss.png".format(bottle_nodes)
# plt.savefig(location+title)

In [ ]:
transformer.save("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/transformer.h5")

In [ ]:
transformer = tf.keras.models.load_model("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/transformer.h5")

In [ ]:
# Freeze weights of autoencoder
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
for k,v in autoencoder._get_trainable_state().items():
    k.trainable = False
autoencoder.compile(optimizer=optimizer, loss="mse")

for k,v in encoder._get_trainable_state().items():
    k.trainable = False
encoder.compile(optimizer=optimizer, loss="mse")

for k,v in decoder._get_trainable_state().items():
    k.trainable = False
decoder.compile(optimizer=optimizer, loss="mse")


# LSTM model
input = Input(shape=(sequence_length-1, bottle_nodes,))
lstm = LSTM(512, activation='relu', return_sequences=True)(input)
lstm = LSTM(512, activation='relu', return_sequences=True)(lstm)
lstm = LSTM(512, activation='relu', return_sequences=True)(lstm)
lstm = LSTM(512, activation='relu', return_sequences=False)(lstm)

dropout = Dropout(0.5)(lstm)



linear = Dense(bottle_nodes)(dropout)

lstm = tf.keras.models.Model(input, linear)
lstm.compile(loss="mse", optimizer=optimizer)
lstm.summary()

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)
history = lstm.fit(x_train, y_train, epochs=40, verbose=1, validation_data=(x_valid, y_valid), callbacks=[callback])

In [ ]:
# epochs = history.epoch
num_epochs = np.linspace(1,40,40)
train_loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure()
plt.plot(num_epochs, train_loss, label="Training loss")
plt.plot(num_epochs, val_loss, label="Validation loss")
plt.legend(loc="upper right", fontsize=10)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.xlim([num_epochs[0], num_epochs[-1]])
title = "loss.png".format(bottle_nodes)
# plt.savefig(location+title)

In [ ]:
lstm.save("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/lstm.h5")

In [ ]:
lstm = tf.keras.models.load_model("/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/lstm.h5")

In [ ]:
seq_i = 4
true_encoded = tf.expand_dims(encoder_labels[seq_i], 0)
true_decoded = decoder.predict(true_encoded, verbose=0)

# # Transformer
# seq_encoded = tf.expand_dims(encoder_outs[seq_i], 0)
# pred_encoded = transformer.predict(seq_encoded, verbose=0)
# pred_decoded = decoder.predict(pred_encoded, verbose=0)

# LSTM
seq_encoded = tf.expand_dims(encoder_outs[seq_i], 0)
pred_encoded = lstm.predict(seq_encoded, verbose=0)
pred_decoded = decoder.predict(pred_encoded, verbose=0)

plt.figure()
plt.imshow(true_decoded[0])

plt.figure()
plt.imshow(pred_decoded[0])

In [ ]:
seq_len = 110

IMAGE_SIZE = 64
sequence_length = 10  # Number of frames per sequence
MIN_SIZE = 0.5e-6
MAX_SIZE = 1.5e-6
MAX_VEL = 10  # Maximum velocity. The higher the trickier!
MAX_PARTICLES = 3  # Max number of particles in each sequence. The higher the trickier!

# Defining properties of the particles
particle = dt.Sphere(
    intensity=lambda: 10 + 10 * np.random.rand(),
    radius=lambda: MIN_SIZE + np.random.rand() * (MAX_SIZE - MIN_SIZE),
    position=lambda: IMAGE_SIZE * np.random.rand(2),
    vel=lambda: MAX_VEL * np.random.rand(2),
    position_unit="pixel",
)

# Defining an update rule for the particle position
def get_position(previous_value, vel):

    newv = previous_value + vel
    for i in range(2):
        if newv[i] > 63:
            newv[i] = 63 - np.abs(newv[i] - 63)
            vel[i] = -vel[i]
        elif newv[i] < 0:
            newv[i] = np.abs(newv[i])
            vel[i] = -vel[i]
    return newv


particle = dt.Sequential(particle, position=get_position)

# Defining properties of the microscope
optics = dt.Fluorescence(
    NA=1,
    output_region=(0, 0, IMAGE_SIZE, IMAGE_SIZE),
    magnification=10,
    resolution=(1e-6, 1e-6, 1e-6),
    wavelength=633e-9,
)

# Combining everything into a dataset.
# Note that the sequences are flipped in different directions, so that each unique sequence defines
# in fact 8 sequences flipped in different directions, to speed up data generation
sequential_images = dt.Sequence(
    optics(particle ** (lambda: 1 + np.random.randint(MAX_PARTICLES))),
    sequence_length=seq_len,
)
dataset_b2 = sequential_images >> dt.FlipUD() >> dt.FlipDiagonal() >> dt.FlipLR()

In [ ]:
encoder_out_list = []
encoder_label_list = []

for seq in dataset_b2:
  true_seq = tf.convert_to_tensor(seq).numpy()
  for i in range(true_seq.shape[0]):
    true_seq[i] = true_seq[i] / np.max(true_seq[i])
  break

plt.figure()
plt.imshow(true_seq[9])

In [ ]:
seq_encoded = encoder.predict(true_seq[:9], verbose=0)
pred_list = []
true_list = []

for i in range(seq_len):
  seq_encoded = tf.expand_dims(seq_encoded, 0)

  # pred_encoded = transformer.predict(seq_encoded, verbose=0) # Transformer
  pred_encoded = lstm.predict(seq_encoded, verbose=0) # LSTM

  pred_decoded = decoder.predict(pred_encoded, verbose=0)
  pred_decoded = pred_decoded / np.max(pred_decoded)
  pred_list.append(pred_decoded[0])
  pred_encoded_next = encoder.predict(pred_decoded, verbose=0)
  seq_encoded = np.vstack([seq_encoded[0], pred_encoded_next[0]])
  seq_encoded = np.delete(seq_encoded, (0), axis=0)

In [ ]:
y = 0
plt.figure()
plt.imshow(true_seq[9+y])

plt.figure()
plt.imshow(pred_list[y])

In [ ]:
num_encoders = 3
num_att_gates = 3
num_lstms = 4
version = 6
num_frames = seq_len - 10 - 40

# location = "/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/3b/encoders={}_gates={}_v{}/".format(num_encoders, num_att_gates, version) # Transformer
location = "/content/drive/MyDrive/Skola/CAS/Advanced Machine Learning/HW_B/3c/lstms={}_v{}/".format(num_lstms, version) # LSTM

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 5))

for y in range(num_frames):
  ax1.imshow(pred_list[y])
  ax1.set_title('Pred.')

  ax2.imshow(true_seq[9+y])
  ax2.set_title('True')

  fig.suptitle('Frame {}'.format(y+1))

  title = 'frame={}'.format(y+1)
  plt.savefig(location+title)

In [ ]:
from PIL import Image
import os

# create a list of all image files in the directory
image_files = [location + "/" + file for file in os.listdir(location) if file.endswith(".png")]

# create a list of image objects from the image files
images = [Image.open(file) for file in image_files]

# create a GIF from the image sequence
images[0].save(location+'sim.gif', save_all=True, append_images=images[1:], optimize=False, duration=200, loop=0)